##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Entornos

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/2_environments_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/2_environments_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/2_environments_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/agents/tutorials/2_environments_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

## Introducción

El objetivo del Aprendizaje por Refuerzo (RL) es diseñar agentes que aprendan mediante la interacción con un entorno. En la configuración estándar del RL, el agente recibe una observación en cada paso de tiempo y elige una acción. La acción se aplica al entorno y éste devuelve una recompensa y una nueva observación. El agente entrena una política para elegir acciones que maximicen la suma de recompensas, también conocida como rendimiento.

En TF-Agents, los entornos se pueden implementar tanto en Python como en TensorFlow. Los entornos Python suelen ser más fáciles de implementar, entender y depurar, pero los entornos TensorFlow son más eficientes y permiten una paralelización natural. El flujo de trabajo más común consiste en implementar un entorno en Python y utilizar una de nuestras envolturas para convertirlo automáticamente en TensorFlow.

Primero, veamos los entornos Python. Los entornos TensorFlow siguen una API muy similar.

## Preparación


Si todavía no ha instalado tf-agents o gym, ejecute los siguientes comandos:

In [ ]:
!pip install tf-agents[reverb]


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

## Entornos de Python

Los entornos Python tienen un método `step(action) -> next_time_step` que aplica una acción al entorno y devuelve la siguiente información sobre el paso posterior:

1. `observation`: esta es la parte del estado del entorno que el agente puede observar para elegir sus acciones para el siguiente paso.
2. `reward`: el agente está aprendiendo a maximizar la suma de estas recompensas a través de múltiples pasos.
3. `step_type`: las interacciones con el entorno suelen formar parte de una secuencia o episodio. Por ejemplo, varios movimientos en un juego de ajedrez. step_type puede ser `FIRST`, `MID` o `LAST` para indicar si este paso de tiempo es el primero, uno intermedio o el último de una secuencia.
4. `discount`: este es un flotante que representa cuánto debe ponderarse la recompensa en el siguiente paso de tiempo respecto a la recompensa en el paso de tiempo actual.

Estos se agrupan en una tupla nombrada `TimeStep(step_type, reward, discount, observation)`.

La interfaz que deben implementar todos los entornos Python se encuentra en `environments/py_environment.PyEnvironment`. Estos son los métodos principales:

In [ ]:
class PyEnvironment(object):

  def reset(self):
    """Return initial_time_step."""
    self._current_time_step = self._reset()
    return self._current_time_step

  def step(self, action):
    """Apply action and return new time_step."""
    if self._current_time_step is None:
        return self.reset()
    self._current_time_step = self._step(action)
    return self._current_time_step

  def current_time_step(self):
    return self._current_time_step

  def time_step_spec(self):
    """Return time_step_spec."""

  @abc.abstractmethod
  def observation_spec(self):
    """Return observation_spec."""

  @abc.abstractmethod
  def action_spec(self):
    """Return action_spec."""

  @abc.abstractmethod
  def _reset(self):
    """Return initial_time_step."""

  @abc.abstractmethod
  def _step(self, action):
    """Apply action and return new time_step."""

Además del método `step()`, los entornos también proporcionan un método `reset()` que inicia una nueva secuencia y ofrece un `TimeStep` inicial. No es necesario llamar al método `reset` explícitamente. Se asume que los entornos se reinician automáticamente, ya sea cuando llegan al final de un episodio o cuando se llama a step() por primera vez.

Tenga en cuenta que las subclases no implementan `step()` ni `reset()` de forma directa. En lugar de eso, anulan los métodos `_step()` y `_reset()`. Los pasos de tiempo que devuelven estos métodos serán almacenados en caché y expuestos a través de `current_time_step()`.

Los métodos `observation_spec` y `action_spec` devuelven un nido de `(Bounded)ArraySpecs` que describe el nombre, la forma, el tipo de datos y los intervalos de las observaciones y las acciones respectivamente.

En TF-Agents, constantemente hablamos de nidos, que se definen como una estructura con forma de árbol que está compuesta de listas, tuplas, tuplas nombradas o diccionarios. Estos pueden estar compuestos arbitrariamente para mantener la estructura de las observaciones y acciones. Esto es muy útil para entornos más complejos en los que hay muchas observaciones y acciones.

### Cómo usar entornos estándar

TF Agents ha incorporado envoltorios para muchos entornos estándar como OpenAI Gym, DeepMind-control y Atari, para que sigan nuestra interfaz `py_environment.PyEnvironment`. Estos entornos envueltos se pueden cargar fácilmente con nuestros paquetes de entornos. Carguemos el entorno CartPole desde OpenAI gym y veamos la acción y time_step_spec.

In [ ]:
environment = suite_gym.load('CartPole-v0')
print('action_spec:', environment.action_spec())
print('time_step_spec.observation:', environment.time_step_spec().observation)
print('time_step_spec.step_type:', environment.time_step_spec().step_type)
print('time_step_spec.discount:', environment.time_step_spec().discount)
print('time_step_spec.reward:', environment.time_step_spec().reward)


De este modo vemos que el entorno espera acciones de tipo `int64` en [0, 1] y devuelve `TimeSteps` donde las observaciones son un vector `float32` de longitud 4 y el factor de descuento es un `float32` en [0.0, 1.0]. Ahora, intentemos adoptar una acción fija `(1,)` para todo un episodio.

In [ ]:
action = np.array(1, dtype=np.int32)
time_step = environment.reset()
print(time_step)
while not time_step.is_last():
  time_step = environment.step(action)
  print(time_step)

### Cómo crear su propio entorno de Python

Para muchos clientes, un caso de uso común es aplicar uno de los agentes estándar (ver agentes/) en TF-Agents a su problema. Para hacer esto, tienen que enmarcar su problema como un entorno. Veamos cómo implementar un entorno en Python.

Supongamos que queremos entrenar a un agente para que juegue al siguiente juego de cartas (inspirado en el Black Jack):

1. El juego se juega con una baraja infinita de cartas numeradas del 1 al 10.
2. En cada turno, el agente puede hacer 2 cosas: obtener una nueva carta al azar o detener la ronda en curso.
3. El objetivo es conseguir que la suma de las cartas se acerque lo más posible a 21 al final de la ronda, sin pasarse.

Un entorno que representa un juego podría verse de esta forma:

1. Acciones: tenemos 2 acciones. Acción 0: pedir una nueva carta, y Acción 1: finalizar la ronda en curso.
2. Observaciones: suma de las cartas en la ronda actual.
3. Recompensa: el objetivo es acercarse lo más posible a 21 sin pasarse, por lo que podemos lograrlo al usar la siguiente recompensa al final de la ronda: sum_of_cards - 21 if sum_of_cards &lt;= 21, else -21


In [ ]:
class CardGameEnv(py_environment.PyEnvironment):

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,), dtype=np.int32, minimum=0, name='observation')
    self._state = 0
    self._episode_ended = False

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = 0
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.int32))

  def _step(self, action):

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # Make sure episodes don't go on forever.
    if action == 1:
      self._episode_ended = True
    elif action == 0:
      new_card = np.random.randint(1, 11)
      self._state += new_card
    else:
      raise ValueError('`action` should be 0 or 1.')

    if self._episode_ended or self._state >= 21:
      reward = self._state - 21 if self._state <= 21 else -21
      return ts.termination(np.array([self._state], dtype=np.int32), reward)
    else:
      return ts.transition(
          np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)

Veamos si hemos hecho todo correctamente al definir el entorno anterior. Al crear un entorno propio hay que asegurarse de que las observaciones y los time_steps generados sigan las formas y tipos correctos tal y como se definen en las especificaciones. Estos se utilizan para generar el gráfico TensorFlow y por lo tanto pueden generar problemas difíciles de depurar si nos equivocamos.

Para validar nuestro entorno usaremos una política aleatoria para generar acciones e iteraremos durante 5 episodios para asegurarnos de que las cosas funcionen según lo previsto. Se producirá un error si recibimos un time_step que no siga las especificaciones del entorno.

In [ ]:
environment = CardGameEnv()
utils.validate_py_environment(environment, episodes=5)

Ahora que sabemos que el entorno funciona según lo previsto, ejecutémoslo con una política fija: pedir 3 cartas y terminar la ronda.

In [ ]:
get_new_card_action = np.array(0, dtype=np.int32)
end_round_action = np.array(1, dtype=np.int32)

environment = CardGameEnv()
time_step = environment.reset()
print(time_step)
cumulative_reward = time_step.reward

for _ in range(3):
  time_step = environment.step(get_new_card_action)
  print(time_step)
  cumulative_reward += time_step.reward

time_step = environment.step(end_round_action)
print(time_step)
cumulative_reward += time_step.reward
print('Final Reward = ', cumulative_reward)

### Envoltorios de entorno

Un envoltorio de entorno toma un entorno Python y devuelve una versión modificada del entorno. Tanto el entorno original como el modificado son instancias de `py_environment.PyEnvironment`, y se pueden encadenar varios envoltorios.

Algunos de los envoltorios comunes se pueden encontrar en `environments/wrappers.py`. Por ejemplo:

1. `ActionDiscretizeWrapper`: convierte un espacio de acción continuo en un espacio de acción discreto.
2. `RunStats`: captura las estadísticas de ejecución del entorno, como el número de pasos dados, el número de episodios completados, etc.
3. `TimeLimit`: finaliza el episodio tras un número fijo de pasos.


#### Ejemplo 1: Action Discretize Wrapper

InvertedPendulum es un entorno PyBullet que acepa acciones continuas en el intervalo `[-2, 2]`. Si queremos entrenar un agente de acción discreto como DQN en este entorno, debemos discretizar (cuantificar) el espacio de acción. Esto es exactamente lo que hace `ActionDiscretizeWrapper`. Compare `action_spec` antes y después de aplicar el envoltorio:

In [ ]:
env = suite_gym.load('Pendulum-v1')
print('Action Spec:', env.action_spec())

discrete_action_env = wrappers.ActionDiscretizeWrapper(env, num_actions=5)
print('Discretized Action Spec:', discrete_action_env.action_spec())

`discrete_action_env` envuelto es una instancia de `py_environment.PyEnvironment` y se puede tratar como un entorno de Python regular.


## Entornos de TensorFlow

La interfaz para entornos TF se define en `environments/tf_environment.TFEnvironment` y tiene un aspecto muy similar al de los entornos Python. Los entornos TF son distintos a los entornos Python en algunos sentidos:

- Generan objetos tensoriales en lugar de arreglos.
- Los entornos TF agregan una dimensión de lote a los tensores generados cuando se comparan con las especificaciones.

Convertir entornos Python en entornos TF le permite a tensorflow paralelizar las operaciones. Por ejemplo, podríamos definir una `collect_experience_op` que recopile datos del entorno y los agregue a un `replay_buffer`, y una `train_op` que lea ese `replay_buffer` y entrene al agente, y ejecutarlas en paralelo con total naturalidad en TensorFlow.

In [ ]:
class TFEnvironment(object):

  def time_step_spec(self):
    """Describes the `TimeStep` tensors returned by `step()`."""

  def observation_spec(self):
    """Defines the `TensorSpec` of observations provided by the environment."""

  def action_spec(self):
    """Describes the TensorSpecs of the action expected by `step(action)`."""

  def reset(self):
    """Returns the current `TimeStep` after resetting the Environment."""
    return self._reset()

  def current_time_step(self):
    """Returns the current `TimeStep`."""
    return self._current_time_step()

  def step(self, action):
    """Applies the action and returns the new `TimeStep`."""
    return self._step(action)

  @abc.abstractmethod
  def _reset(self):
    """Returns the current `TimeStep` after resetting the Environment."""

  @abc.abstractmethod
  def _current_time_step(self):
    """Returns the current `TimeStep`."""

  @abc.abstractmethod
  def _step(self, action):
    """Applies the action and returns the new `TimeStep`."""

El método `current_time_step()` devuelve el time_step actual e inicializa el entorno según sea necesario.

El método `reset()` fuerza un restablecimiento del entorno y devuelve el current_step.

Si la `action` no depende del `time_step` se requiere una `tf.control_dependency` en el modo `Graph`.

Por el momento, veamos cómo se crean los `TFEnvironments`.

### Cómo crear su propio entorno de TensorFlow

Esto es más complicado que crear entornos en Python, así que no lo trataremos en este colab. [Aquí](https://github.com/tensorflow/agents/blob/master/tf_agents/environments/tf_environment_test.py) puede ver un ejemplo. El caso de uso más común es implementar un entorno en Python y usar el envoltorio `TFPyEnvironment` para envolverlo en TensorFlow (se explica a continuación).

### Cómo envolver un entorno Python en TensorFlow

Podemos envolver fácilmente un entorno Python en un entorno TensorFlow gracias a este envoltorio `TFPyEnvironment`.

In [ ]:
env = suite_gym.load('CartPole-v0')
tf_env = tf_py_environment.TFPyEnvironment(env)

print(isinstance(tf_env, tf_environment.TFEnvironment))
print("TimeStep Specs:", tf_env.time_step_spec())
print("Action Specs:", tf_env.action_spec())

Observe que ahora son especificaciones de tipo: `(Bounded)TensorSpec`.

### Ejemplos de uso

#### Ejemplo simple

In [ ]:
env = suite_gym.load('CartPole-v0')

tf_env = tf_py_environment.TFPyEnvironment(env)
# reset() creates the initial time_step after resetting the environment.
time_step = tf_env.reset()
num_steps = 3
transitions = []
reward = 0
for i in range(num_steps):
  action = tf.constant([i % 2])
  # applies the action and returns the new TimeStep.
  next_time_step = tf_env.step(action)
  transitions.append([time_step, action, next_time_step])
  reward += next_time_step.reward
  time_step = next_time_step

np_transitions = tf.nest.map_structure(lambda x: x.numpy(), transitions)
print('\n'.join(map(str, np_transitions)))
print('Total reward:', reward.numpy())

#### Episodios completos

In [ ]:
env = suite_gym.load('CartPole-v0')
tf_env = tf_py_environment.TFPyEnvironment(env)

time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
  episode_reward = 0
  episode_steps = 0
  while not time_step.is_last():
    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

print('num_episodes:', num_episodes, 'num_steps:', num_steps)
print('avg_length', avg_length, 'avg_reward:', avg_reward)